In [25]:
import pandas as pd
import os

In [26]:
low_level_func_dir = '/mydata/rbachkaniwala3/code/rajveerb-ml-pipeline-benchmark/code/image_classification/analysis/low_level_func/logs'

In [27]:
# recursively get all files in the directory

op_to_df = {}
for root, d, files in os.walk(low_level_func_dir):
    if files != []:
        root = root.rstrip('/')
        root_dir = root.split('/')[-1]
        print(root_dir)
        # create an empty dataframe
        df = pd.DataFrame()
        for file in files:
            file_path = os.path.join(root, file)
            # read the file as a dataframe
            temp_df = pd.read_csv(file_path)
            # concatenate the dataframe to the df dataframe
            df = pd.concat([df, temp_df])
        
        # add the dataframe to the dictionary
        op_to_df[root_dir] = df

RandomResizedCrop
Collation
convertRGB
ToTensor
Normalize
RandomHorizontalFlip


In [28]:
total_experiments = 5
sampling_interval = 0.01 # in ms
lowest = None
# get all with count < 2
count_more_than_10 = []
# loop through the op_to_df dictionary and generate a count column for how many times the same function in 'Function' column appears
for op, df in op_to_df.items():
    # print(op)
    op_to_df[op]['count'] = op_to_df[op].groupby('Function')['Function'].transform('count')
    # also get average time for each function
    op_to_df[op]['avg_time'] = op_to_df[op].groupby('Function')['CPU Time'].transform('mean')
    # keep only Function and count columns
    op_to_df[op] = op_to_df[op][['Function', 'count','avg_time']]
    # remove redundant rows
    op_to_df[op] = op_to_df[op].drop_duplicates()
    
    # iterate through the rows and print the function and count
    for index, row in op_to_df[op].iterrows():

        # if function takes more than 10 ms on average, then it is will be seen every experiment
        if float(row['avg_time']) > sampling_interval:
            probability = 1
        else:
            probability = float(row['avg_time']) / sampling_interval
        probability_of_seeing = 100*(1-(1-probability)**(total_experiments))

        if int(row['count']) > 10:
            count_more_than_10.append(f"Operation: {op}, Function: {row['Function']}, avg_time: {row['avg_time']*1000} ms, probability_of_seeing: {probability_of_seeing}%")
        if float(row['avg_time'])*1000 < 1:
            # prints if avg_time is less than 1 ms
            print('\t',row['Function'], row['count'], row['avg_time']*1000,'ms','\n\t\t', probability_of_seeing,'%')

        if lowest is None or probability < lowest[0]:
            lowest = [probability_of_seeing, row['Function'], row['avg_time']]


	 func@0x1e7f80 1 0.477 ms 
		 21.680677554733595 %
	 pymalloc_alloc 1 0.224 ms 
		 10.709354106400525 %
	 c10::impl::OperatorEntry::lookup 1 0.756 ms 
		 32.50063549600507 %
	 __GI___pthread_mutex_lock 1 0.627 ms 
		 27.65757126557541 %
	 __GI___libc_malloc 1 0.948 ms 
		 39.22531361811708 %
	 __libc_calloc 1 0.74 ms 
		 31.91445261266238 %


In [29]:
print('Lowest probability:', lowest[0], '% for function:', lowest[1], 'with avg_time:', lowest[2] *1000, 'ms')

Lowest probability: 31.91445261266238 % for function: __libc_calloc with avg_time: 0.74 ms


In [30]:
# loop through the count_less_than_2 list and print the function and count
print('Functions with count > 10:')
for item in count_more_than_10:
    print(item)

Functions with count > 10:
Operation: RandomResizedCrop, Function: ImagingResampleHorizontal_8bpc, avg_time: 414.09835 ms, probability_of_seeing: 100%
Operation: RandomResizedCrop, Function: __memmove_avx_unaligned_erms, avg_time: 110.0085 ms, probability_of_seeing: 100%
Operation: RandomResizedCrop, Function: _int_free, avg_time: 20.0 ms, probability_of_seeing: 100%
Operation: RandomResizedCrop, Function: ImagingResampleVertical_8bpc, avg_time: 9.713642857142856 ms, probability_of_seeing: 99.99999807451516%
Operation: Collation, Function: c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::native::AVX2::VectorizedLoop2d<at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(float)#1}, at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(at::vec::AVX2::Vectorized<float>)#2}>>, avg